In [31]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

import pandas as pd
import glob

In [32]:
path = "arkanoid/log"
filenames = glob.glob(path + "/*.pickle")
table = pd.DataFrame()
for filename in filenames:
    data = pd.read_pickle(filename)
    df = pd.DataFrame(data["ml"]['scene_info'],  data["ml"]['command'])
    df.rename(columns={'index': 'command'}, inplace=True)
    df.drop(df.tail(1).index,inplace=True)
    table = table.append(df)

In [33]:
# table.loc[table['command'] == "NONE", 'command'] = 0
# table.loc[table['command'] == "MOVE_LEFT", 'command'] = 1
# table.loc[table['command'] == "MOVE_RIGHT", 'command'] = 2
# table.loc[table['command'] == "SERVE_TO_LEFT", 'command'] = 0
# table.loc[table['command'] == "SERVE_TO_RIGHT", 'command'] = 0
# table.command.fillna(0, inplace = True)

In [34]:
table

,frame,status,ball,platform,bricks,hard_bricks
SERVE_TO_LEFT,0,GAME_ALIVE,"(93, 395)","(75, 400)","[(50, 60), (75, 60), (100, 60), (125, 60), (60...","[(35, 50), (60, 50), (85, 50), (110, 50), (135..."
NONE,2,GAME_ALIVE,"(86, 388)","(75, 400)","[(50, 60), (75, 60), (100, 60), (125, 60), (60...","[(35, 50), (60, 50), (85, 50), (110, 50), (135..."
NONE,3,GAME_ALIVE,"(79, 381)","(75, 400)","[(50, 60), (75, 60), (100, 60), (125, 60), (60...","[(35, 50), (60, 50), (85, 50), (110, 50), (135..."
NONE,4,GAME_ALIVE,"(72, 374)","(75, 400)","[(50, 60), (75, 60), (100, 60), (125, 60), (60...","[(35, 50), (60, 50), (85, 50), (110, 50), (135..."
NONE,5,GAME_ALIVE,"(65, 367)","(75, 400)","[(50, 60), (75, 60), (100, 60), (125, 60), (60...","[(35, 50), (60, 50), (85, 50), (110, 50), (135..."
...,...,...,...,...,...,...
NONE,1827,GAME_ALIVE,"(90, 42)","(0, 400)","[(35, 70)]",[]
NONE,1828,GAME_ALIVE,"(83, 49)","(0, 400)","[(35, 70)]",[]
NONE,1829,GAME_ALIVE,"(76, 56)","(0, 400)","[(35, 70)]",[]
NONE,1830,GAME_ALIVE,"(69, 63)","(0, 400)","[(35, 70)]",[]


In [35]:
table['ball_x'] = table['ball'].str[0]
table['ball_y'] = table['ball'].str[1]
table['platform_left'] = table['platform'].str[0]
table['platform_right']  = table['platform_left'] + 40

In [36]:
#table['from_right_wall'] = 195 - table['ball_x']
#table['from_bottom'] = 400 - table['ball_y']

In [37]:
temp_list = table['ball_x'].tolist()
temp_list.pop()
temp_list.insert(0,93)
table['ball_x_last1'] = temp_list
temp_list.pop()
temp_list.insert(0,93)
table['ball_x_last2'] = temp_list
temp_list.pop()
temp_list.insert(0,93)
table['ball_x_last3'] = temp_list

temp_list = table['ball_y'].tolist()
temp_list.pop()
temp_list.insert(0,395)
table['ball_y_last1'] = temp_list
temp_list.pop()
temp_list.insert(0,395)
table['ball_y_last2'] = temp_list
temp_list.pop()
temp_list.insert(0,395)
table['ball_y_last3'] = temp_list

In [38]:
table['ball_vector_x'] = table['ball_x'] - table['ball_x_last1']
table['ball_vector_y'] = table['ball_y'] - table['ball_y_last1']


In [39]:
#table['left_diff'] = table['ball_x'] - table['platform_left']
#table['right_diff'] = table['ball_x'] - table['platform_right']

In [40]:
#del table['frame']
del table['bricks']
del table['hard_bricks']
del table['ball']
del table['platform']
#del table['platform_left']
#del table['platform_right']

# del table['ball_x_last1']
# del table['ball_y_last1']
del table['ball_x_last2']
del table['ball_x_last3']
del table['ball_y_last2']
del table['ball_y_last3']
# del table['frame']
del table['status']

In [41]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(table.drop(columns='platform_left') , table['platform_left'], test_size=0.3, random_state=9)
#參數區間
param_grid = {'n_neighbors':[1, 3 ,5, 7, 9, 13]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10)#, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('arkanoid/ml/arkanoid_n3_20210322_knn_model.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV 1/2; 1/6] START n_neighbors=1...............................................
[CV 1/2; 1/6] END .............................n_neighbors=1; total time=   0.3s
[CV 2/2; 1/6] START n_neighbors=1...............................................
[CV 2/2; 1/6] END .............................n_neighbors=1; total time=   0.3s
[CV 1/2; 2/6] START n_neighbors=3...............................................
[CV 1/2; 2/6] END .............................n_neighbors=3; total time=   0.3s
[CV 2/2; 2/6] START n_neighbors=3...............................................
[CV 2/2; 2/6] END .............................n_neighbors=3; total time=   0.3s
[CV 1/2; 3/6] START n_neighbors=5...............................................
[CV 1/2; 3/6] END .............................n_neighbors=5; total time=   0.3s
[CV 2/2; 3/6] START n_neighbors=5...............................................
[CV 2/2; 3/6] END .............................n_

In [42]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 1}
[[ 672    5    0 ...    0    0    0]
 [   8  145    5 ...    0    0    0]
 [   0    0  167 ...    0    0    0]
 ...
 [   0    0    0 ...  426    5    2]
 [   0    0    0 ...    6  258   14]
 [   0    0    0 ...    0    4 1292]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       677
           5       0.97      0.92      0.94       158
          10       0.87      0.77      0.82       217
          15       0.83      0.88      0.85       250
          20       0.83      0.87      0.85       214
          25       0.92      0.83      0.87       413
          30       0.86      0.97      0.91       392
          35       0.90      0.93      0.92       286
          40       0.89      0.79      0.83       159
          45       0.97      0.98      0.97       582
          50       0.91      0.95      0.93       476
          55       0.97      0.94      0.95       884
          60       0.94      0.97      0.95       

In [43]:
table.drop(columns='platform_left').head()

,frame,ball_x,ball_y,platform_right,ball_x_last1,ball_y_last1,ball_vector_x,ball_vector_y
SERVE_TO_LEFT,0,93,395,115,93,395,0,0
NONE,2,86,388,115,93,395,-7,-7
NONE,3,79,381,115,86,388,-7,-7
NONE,4,72,374,115,79,381,-7,-7
NONE,5,65,367,115,72,374,-7,-7


In [44]:
table.drop(columns='platform_left').columns

Index(['frame', 'ball_x', 'ball_y', 'platform_right', 'ball_x_last1',
       'ball_y_last1', 'ball_vector_x', 'ball_vector_y'],
      dtype='object')